## 3 - Setting Up the RL Agent Training Environment

In this section, I go through a few different potential playing strategies for blackjack and offer an analysis on each of them. Namely, the 3 strategies I will test are a Discrete Standing Strategy, a Stochastic Standing Strategy, and probably the most well-known and mathematically proven strategy known as of today, the Basic Strategy. (Another more complex strategy based off of Basic Strategy involves a concept called "card counting" which I will not be showing.)

take the stuff from gym.ipynb on how i set up this env and also no splitting in this environment, maybe add one for splitting later